In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import os
os.chdir("/content/drive/My Drive/mercari_capstone2")
!ls -l

total 1713868
-rw------- 1 root root    2441360 Jun 23 10:05  1-EDA.ipynb
-rw------- 1 root root      40868 Jun 25 11:18 '2-featureization and pickleing.ipynb'
-rw------- 1 root root      51452 Jun 25 11:10  3_Hyperparameter_tuning.ipynb
-rw------- 1 root root      49476 Jun 25 11:07 '4-train deploy.ipynb'
-rw------- 1 root root     122880 Jun 24 19:02  Mercari_to1.db
drwx------ 2 root root       4096 Jun 23 11:42  pickle
-rw------- 1 root root   11853920 Jun 25 11:18  price_log.pickle
-rw------- 1 root root 1402624694 Jun 25 11:18  tfidf.pickle
-rw------- 1 root root  337809843 Nov 11  2017  train.tsv


In [15]:
#importing modules/libraries
import pandas as pd
import numpy as np
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
import gc
import sys
import os
import psutil
# from scipy.stats import randint as sp_randint
# from scipy.stats import uniform as sp_uniform


from tqdm.notebook import tqdm
# from collections import Counter
# from collections import defaultdict
import re
import random
# from random import sample
# from bs4 import BeautifulSoup
import pickle
import inspect
import time

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.linear_model import Lasso,Ridge

import warnings
warnings.filterwarnings("ignore")

In [16]:
# defining root mean square error over Log transformed y_test data
# (as linear models homoscedasticity can be kept in check for better prediction)
# and hence an effective Root Mean Square Log Error
def error(y_test, predictions):
  return np.sqrt(mean_squared_error( y_test, predictions ))

In [17]:
tr_len = 1185329# demarkation of cv data(0.8 percent)
whole_tr = 1481661# whole train data

In [18]:
# loading prepared data
with open('tfidf.pickle','rb') as f:
  df=pickle.load(f)
with open('price_log.pickle','rb') as f:
  y=pickle.load(f)

In [22]:
df = df.tocsr()
df = df[:whole_tr] # only taking train and cv data
gc.collect()

8

In [23]:
np.isnan(df.data).sum()

0

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(df,y, train_size = round(0.8*df.shape[0]))

In [ ]:
#alpha over large range
para = {'alpha': [0.001,0.01,0.1,1,2,5,10]}
clf= Ridge(max_iter=10000, tol=0.0005, solver='auto', random_state=34)
results = GridSearchCV(clf,para, cv=3, verbose=5,n_jobs=-1,scoring='neg_mean_squared_error',return_train_score=True)
results.fit(X_train,Y_train)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


GridSearchCV(cv=3, estimator=Ridge(max_iter=10000, random_state=34, tol=0.0005),
             n_jobs=-1, param_grid={'alpha': [0.001, 0.01, 0.1, 1, 2, 5, 10]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=5)

In [ ]:
print('Best score reached: {} with params: {} '.format(results.best_score_, results.best_params_))#gscv

Best score reached: -0.20260475779543588 with params: {'alpha': 5} 


In [ ]:
# zooming in the range from above parameter output this cell ahs been run multiple of times
# with different values one zooming more after another
para = {'alpha': np.linspace(4,6,5)}
clf= Ridge(max_iter=10000, tol=0.0005, solver='auto', random_state=34)
results = GridSearchCV(clf,para, cv=3, verbose=5,n_jobs=-1,scoring='neg_mean_squared_error',return_train_score=True)
results.fit(X_train,Y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


GridSearchCV(cv=3, estimator=Ridge(max_iter=10000, random_state=34, tol=0.0005),
             n_jobs=-1, param_grid={'alpha': array([4. , 4.5, 5. , 5.5, 6. ])},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=5)

In [ ]:
print('Best score reached: {} with params: {} '.format(results.best_score_, results.best_params_))#gscv

Best score reached: -0.20260475779543588 with params: {'alpha': 5.0} 


In [ ]:
model= results.best_estimator_

model.fit(X_train, Y_train)
Y_pred = model.predict(X_train)

print('train error {}'.format(error(Y_train,Y_pred)))
Y_pred = model.predict(X_test)

print('test error {}'.format(error(Y_test,Y_pred)))

train error 0.4148969253652342
test error 0.4443864614855584


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive/mercari_capstone")
!ls -l

!pip install optuna

Mounted at /content/drive
total 1512298
drwx------ 2 root root      4096 May 31 12:40 Data
-rw------- 1 root root     22216 Jun  2 10:01 Data_Text_Prep.ipynb
-rw------- 1 root root   2019075 May 22 16:38 EDA.ipynb
-rw------- 1 root root     43759 Jun  4 07:49 hyperparameter_tuning.ipynb
-rw------- 1 root root    196608 Jun  3 16:48 Mercari_to3.db
-rw------- 1 root root   9595096 May 31 16:35 price_log.pickle
-rw------- 1 root root 325554396 May 31 10:20 text.pickle
-rw------- 1 root root 873346297 May 31 16:35 tf_idf.pickle
-rw------- 1 root root 337809843 Nov 11  2017 train.tsv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 10.8 MB/s eta 0:00:00


In [ ]:
#https://optuna.readthedocs.io/en/stable/index.html

import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import pickle

import optuna



def obj_func(trial,data, target):

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "n_estimator":trial.suggest_int("n_estimator", 80, 2500),
        "max_depth": trial.suggest_int("max_depth", 10,40),
        "num_leaves": trial.suggest_int("num_leaves", 80, 200),
        # "min_child_samples": trial.suggest_int("min_child_samples", 50, 300),
        # "min_child_weight" : trial.suggest_float('min_child_weight', 0.002, 1.0),
        "subsample_for_bin":trial.suggest_int("subsample_for_bin", 50000, 100000),
        "learning_rate" : trial.suggest_loguniform('learning_rate', 1e-1, 5),
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-5, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-5, 10.0),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.85),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.85),
        # "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),


    }
    tr_len= 1185329
    train_x, valid_x, train_y, valid_y = train_test_split(
                                      data,target,train_size = tr_len
                                      )
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)




    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)


    return np.sqrt(sklearn.metrics.mean_squared_error(valid_y, pred_labels))


if __name__ == "__main__":

  with open('tf_idf.pickle','rb') as f:
    data=pickle.load(f)
  with open('price_log.pickle','rb') as f:
    target=pickle.load(f)


    whole_len = 1481661
    data = data[:whole_len,:]
    target = target[:whole_len]


    study_name = 'Mercari'
    study = optuna.create_study(study_name=study_name,
                                storage='sqlite:///Mercari_to3.db',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
                                )


    study.optimize(lambda trial : obj_func(trial,data, target), n_trials=20 ,gc_after_trial=True)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-06-26 02:37:40,377] Using an existing study with name 'Mercari' instead of creating a new one.
[I 2023-06-26 04:52:36,581] Trial 1 finished with value: 0.5205743934328912 and parameters: {'learning_rate': 0.3406778782263416, 'max_depth': 35, 'num_iterations': 2157, 'num_leaves': 93, 'subsample_for_bin': 83403}. Best is trial 1 with value: 0.5205743934328912.
[I 2023-06-26 06:26:41,701] Trial 2 finished with value: 8.075360048980799e+37 and parameters: {'learning_rate': 3.045486489683093, 'max_depth': 38, 'num_iterations': 2407, 'num_leaves': 103, 'subsample_for_bin': 99553}. Best is trial 1 with value: 0.5205743934328912.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 1965, in _exec_single_context
    self.dialect.do_execute(
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/default.py", line 921, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: disk I/O error

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 70, in _create_scoped_session
    yield session
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/storage.py", line 774, in get_trial
    trial_model = models.TrialModel.find_or_raise_by_id(trial_id, session)
  File "/usr/local/lib/python3.10/dist-packages/optuna/storages/_rdb/models.py", line 245, in find_or_raise_by_id
    trial = query.one_or_none()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/orm/query.py"

In [ ]:
#https://optuna.readthedocs.io/en/stable/index.html

import lightgbm as lgb
import numpy as np
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split
import pickle

import optuna


def obj_func(trial,data, target):

    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "num_iterations":trial.suggest_int("num_iterations", 2000, 2500),
        "max_depth": trial.suggest_int("max_depth", 30,42),
        "num_leaves": trial.suggest_int("num_leaves", 120, 200),
        # "min_child_samples": trial.suggest_int("min_child_samples", 50, 300),
        # "min_child_weight" : trial.suggest_float('min_child_weight', 0.002, 1.0),
        "subsample_for_bin":trial.suggest_int("subsample_for_bin", 50000, 100000),
        "learning_rate" : trial.suggest_loguniform('learning_rate', 0.1, 0.5),
        # "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-5, 10.0),
        # "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-5, 10.0),
        # "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 0.85),
        # "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 0.85),
        # "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),


    }
    tr_len= 1185329
    train_x, valid_x, train_y, valid_y = train_test_split(
                                      data,target,train_size = tr_len
                                      )
    dtrain = lgb.Dataset(train_x, label=train_y)
    dvalid = lgb.Dataset(valid_x, label=valid_y)




    # Add a callback for pruning.
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, "rmse")
    gbm = lgb.train(
        param, dtrain, valid_sets=[dvalid], verbose_eval=False, callbacks=[pruning_callback]
    )

    preds = gbm.predict(valid_x)
    pred_labels = np.rint(preds)


    return np.sqrt(sklearn.metrics.mean_squared_error(valid_y, pred_labels))


if __name__ == "__main__":

  with open('tfidf.pickle','rb') as f:
    data=pickle.load(f)
  with open('price_log.pickle','rb') as f:
    target=pickle.load(f)


    whole_len = 1481661
    data = data[:whole_len,:]
    target = target[:whole_len]


    study_name = 'Mercari'
    study = optuna.create_study(study_name=study_name,
                                storage='sqlite:///Mercari_to3.db',
                                load_if_exists=True,
                                pruner=optuna.pruners.MedianPruner(n_warmup_steps=10), direction="minimize"
                                )


    study.optimize(lambda trial : obj_func(trial,data, target), n_trials=20 ,gc_after_trial=True)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2023-06-04 08:02:04,222] Using an existing study with name 'Mercari' instead of creating a new one.
<ipython-input-4-9ba24bf5134b>:26: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "learning_rate" : trial.suggest_loguniform('learning_rate', 0.1, 0.5),
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.10/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
[I 2023-06-04 08:42

In [25]:
gc.collect()
lgbt =  lgb.LGBMRegressor(boosting_type='gbdt', objective='regression',
                                   random_state=11, n_jobs=4, subsample_for_bin=50269,
                                   learning_rate=0.36386646562142544, num_leaves=190, max_depth=40, n_estimators=2194)
lgbt.fit(X_train, Y_train)

Y_pred = lgbt.predict(X_train)

print('train error {}'.format(error(Y_train,Y_pred)))
Y_pred = lgbt.predict(X_test)

print('test error {}'.format(error(Y_test,Y_pred)))

train error 0.2207222588822965
test error 0.4395201992375062


In [26]:
with open('pickle/models/lbgm_coeffs','wb') as f:
  pickle.dump(lgbt,f)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-0f122a54d614>", line 1, in <cell line: 1>
    with open('pickle/models/lbgm_coeffs','wb') as f:
OSError: [Errno 107] Transport endpoint is not connected: 'pickle/models/lbgm_coeffs'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb